In [1]:
import csv
import pandas as pd
import math
import json
import numpy as np
import random
import matplotlib as plt
import re
import torch
import jieba
import collections
pd.__version__
# 主要针对所有上市公司的 financial ratios、nonfinancial ratios、Linguistic features(MDA analysis)的分析

'0.25.1'

#### 预处理财务欺诈数据，从国泰安下载和处理财务违规数据:公司研究系列-> 审计信息-> 违规信息-> 上市公司财务违规表，1265条，2020/1/1 ~ 2022/12/31

In [ ]:
weigui = pd.read_excel('../data/上市公司财务违规表/AR_FINVIOLATION_raw.xlsx',encoding="utf-8")
weigui.drop([0,1],axis=0,inplace=True)
# 把违规的时间（ViolationYear）处理成 list 存在原处
for i in weigui.index:
    vio_years = weigui.loc[i,'ViolationYear']
    if vio_years is np.nan:
        continue
    if ';' in vio_years:
        vio_years =  sorted(set(vio_years.split(';')) - set(['N/A','']))
    else: # '用,分隔'
        vio_years =  sorted(set(vio_years.split(',')) - set(['N/A','']))
    weigui.loc[i,'ViolationYear'] = json.dumps([int(i) for i in vio_years])

# 把‘Activity’列删除，因为里面有一些换行符
weigui = weigui.drop(['Law','Activity'], axis=1)
weigui.to_csv('../data/上市公司财务违规表/AR_FINVIOLATION.csv',encoding='gb18030',index=None)

2023年5月24日，杜老师在改初稿(Section 5.1 -> Table 2)时，让把最初的违规公司限定在2003年到2021年，因此生成一个 AR_FINVIOLATION_since2003 初始文件

In [9]:
weigui = pd.read_csv('../data/上市公司财务违规表/AR_FINVIOLATION.csv',encoding='gb18030')

drop_before_2003 = []
for i in weigui.index:
    try:
        vio_years = json.loads(weigui.loc[i,'ViolationYear'])
    except:
        continue
    vio_years = [j for j in vio_years if j >= 2003]
    if vio_years == []:
        drop_before_2003.append(i)
    else:
        weigui.loc[i,'ViolationYear'] = json.dumps(vio_years)

weigui.drop(drop_before_2003,axis=0,inplace=True)
weigui.to_csv('../data/上市公司财务违规表/AR_FINVIOLATION_since2003.csv',encoding='gb18030',index=None)


#### 预处理财务重述数据

In [3]:
RestatementObject = {1:'年报', 2:'中报', 3:'季报', 4:'三季报', 5:'审计报告', 6:'财务报告'}
RestatementType = {1: '收入确认', 2: '成本计价', 3: '费用计价', 4: '资产=存货计价', 5: '企业并购', 6: '分立', 7: '证券相关', 8: '重分类问题', 9: '关联交易', 10: '会计政策变更', 11: '股票拆分', 12: '股票红利', 13: '其他'}
ReasonDescription = {1: '企业内部统计错误', 2: '法律或监管要求', 3: '媒体曝光或者是监管机构披露', 4: '审计机构统计错误', 5: '其他', 9: '其他', np.nan:'', 0:''}

In [ ]:
restate = pd.read_excel('../data/财务重述情况表/AR_FINRESTAT_raw.xlsx')
restate.drop([0,1],axis=0,inplace=True)
for i in restate.index:
    restate.loc[i,'RestatementYear'] = int(restate.loc[i,'RestatementYear'].split('-')[0])
    restate.loc[i,'RestatementObject'] = RestatementObject[restate.loc[i,'RestatementObject']]
    restate.loc[i,'RestatementType'] = RestatementType[restate.loc[i,'RestatementType']]
    restate.loc[i,'ReasonDescription'] = ReasonDescription[restate.loc[i,'ReasonDescription']]
restate.to_csv('../data/财务重述情况表/AR_FINRESTAT.csv',encoding='gb18030',index=None)

#### 处理 industry 和 financial ratios
basic_info (financial ratios) 亟待更新，因为里面的财务指标只到2021年，22年的财务指标要等23年4月30日全部上市公司都披露财报以后才行

In [ ]:
industry_dict ={
    # 这四个是CG_CO里没有的
    '金属制品、机械和设备修理业':'工业',
    '其他服务业':'公用事业',
    '居民服务业':'公用事业',
    '管道运输业':'公用事业'
}

CG_CO= pd.read_csv('../data/行业_公司基本情况文件/CG_Co.csv',encoding='utf-8')
for i in range(CG_CO.shape[0]):
    key = CG_CO.loc[i,'Nnindnme']
    value = CG_CO.loc[i,'Indnme']
    industry_dict[key] = value

# 会计信息质量->财务报告信息，包括财务指标、行业、公司规模
# 表格预处理，包括年份的提取，然后行业的进一步分类
basic_info = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_raw.xlsx',encoding="gbk")
basic_info = basic_info.drop(index=[0,1],axis=0)
basic_info = basic_info.reset_index(drop = True)

# 对行业做一个更上层的归类
basic_info['Industry'] = [industry_dict[i] for i in basic_info['IndustryName']]

time = basic_info['EndDate']
years = []
for t in time:
    years.append(t.split('-')[0])
basic_info['EndDate'] = years

# 中文编码的大小：gbk<gb2312<gb18030，一般情况下，gb18030编码都能解析gbk不能解析的编码信息。
basic_info.to_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding='gb18030',index=None)
# 不存为csv文件是因为csv文件会把11次方以上的数值后面的小数抹去

##### 在 AIQ_LCFinIndexY.xlsx 上手动计算计算12个欺诈相关的财务指标

In [2]:
basic_info = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding='gb18030')
basic_info
# 总资产数据，所有公司都有

,Symbol,ShortName,EndDate,AccountsReceivable,TotalCurrentAssets,Inventory,FixedAssets,TotalAssets,TotalCurrentliabilities,TotalLiabilities,...,IG,lev,LEV,OPM,rg,RG,sg,SG,sgee,SGEE
0,1,深发展A,2000,-3.960032e+08,NaN,NaN,1.587091e+09,6.722750e+10,NaN,6.248862e+10,...,NaN,0.929510,NaN,NaN,-3.960032e+08,NaN,0.000000e+00,NaN,NaN,NaN
1,1,深发展A,2001,1.120290e+07,NaN,NaN,1.763257e+09,1.201270e+11,NaN,1.164993e+11,...,NaN,0.969801,1.043347,NaN,1.120290e+07,-0.028290,0.000000e+00,NaN,NaN,NaN
2,1,深发展A,2002,-7.627110e+08,NaN,NaN,2.380640e+09,1.661664e+11,NaN,1.623984e+11,...,NaN,0.977324,1.007757,NaN,-7.627110e+08,-68.081553,0.000000e+00,NaN,NaN,NaN
3,1,深发展A,2003,-8.331532e+08,NaN,NaN,2.388062e+09,1.928510e+11,NaN,1.888859e+11,...,NaN,0.979440,1.002165,NaN,-8.331532e+08,1.092358,0.000000e+00,NaN,NaN,NaN
4,1,深发展A,2004,-8.742457e+08,NaN,NaN,3.243569e+09,2.042864e+11,NaN,1.996018e+11,...,NaN,0.977068,0.997579,NaN,-8.742457e+08,1.049322,0.000000e+00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52569,900957,凌云B股,2017,8.886504e+07,1.975161e+08,NaN,5.383558e+08,1.019428e+09,1.025854e+08,5.955854e+08,...,NaN,0.584235,0.882582,0.305931,8.886504e+07,0.659136,9.811317e+07,1.216833,0.476506,0.735033
52570,900957,凌云B股,2018,1.421202e+08,2.029610e+08,NaN,5.037777e+08,1.007002e+09,9.661402e+07,5.554740e+08,...,NaN,0.551612,0.944161,0.260902,1.421202e+08,1.599281,1.061131e+08,1.081537,0.387669,0.813566
52571,900957,凌云B股,2019,1.779797e+08,2.318127e+08,NaN,4.723017e+08,1.017715e+09,1.179856e+08,5.408456e+08,...,NaN,0.531432,0.963416,0.241292,1.779797e+08,1.252318,1.050223e+08,0.989721,0.396076,1.021687
52572,900957,凌云B股,2020,2.077017e+08,2.468748e+08,NaN,4.380555e+08,1.008562e+09,8.253442e+07,5.098344e+08,...,NaN,0.505506,0.951216,0.219894,2.077017e+08,1.166997,9.940559e+07,0.946519,0.381757,0.963848


In [26]:
# 读取基于国泰安的12个财务指标，然后把每个公司已记录的第一年指标全设为 Nan
finRatio = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY.xlsx',encoding='gb18030')
# 把第一年的指标归零，因为第一年算出来的指标是错的，用的是当前公司的第一年除以上面一家公司的最后一年。

pre_firm = ''
for i in range(finRatio.shape[0]):
    firm = finRatio.loc[i,'Symbol']
    if firm != pre_firm:
        # 把所有的 指标设为 nan
        finRatio.iloc[i,25:] = np.nan
    pre_firm = firm

used_columns = ['Symbol', 'ShortName', 'EndDate', 'TotalAssets', 'OperatingRevenue', 'Industry', 'AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG', 'LEV', 'OPM', 'RG', 'SG', 'SGEE']
finRatio.loc[:,used_columns].to_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_s1.xlsx',index=None,encoding='gb18030')

In [227]:
finRatio = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_s1.xlsx', encoding='gb18030')

第一阶段筛选：\
把全为nan和2000、2001、2022年的行删掉\
2002年的数据先保留一下，在算 R1-P1、R2-P2...的时候有用\
生成 AIQ_LCFinIndexY_padded

In [199]:
finRatio = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_s1.xlsx', encoding='gb18030')

drop_index_1 = []
drop_index_2 = []
# 先把 含有 Nan 的行删掉。
for i in range(finRatio.shape[0]):
    if finRatio.loc[i,:].isnull().tolist().count(True)>=11:
        drop_index_1.append(i)

    elif int(finRatio.loc[i,'EndDate']) in [2000, 2001, 2022]:
        # 2002 年的数据先不删，在 算 R1-P1、R2-P2...的时候有用
        drop_index_2.append(i)

print('共有%d行的数据总资产、行业和12个指标全为空' % len(drop_index_1))
print('共有%d行的数据在2002年以前，且也不是2022年以后' % len(drop_index_2))

finRatio = finRatio.drop(drop_index_1 + drop_index_2, axis=0)
finRatio = finRatio.set_index(['Symbol'])

共有4968行的数据总资产、行业和12个指标全为空
共有1081行的数据在2002年以前，且也不是2022年以后


第一阶段结束后，将一些0值和空值补成1.

In [200]:
def padding_values(df):
    # 可以补齐的一些指标
    padding_cols = ['AQI', 'DSIR','DEPI', 'GMI', 'IG', 'LEV', 'RG', 'SG', 'SGEE']
    df.reset_index(drop=True, inplace=True)

    if df.iloc[:,5:].isnull().sum().sum()/(df.shape[0] * (df.shape[1]-5)) < 0.1:
        # 空值如果超过了 10%，就不用补了，之后直接把这家公司筛了
        for col in padding_cols:
            for i in range(df.shape[0]):
                if df.loc[i, col]==0:
                    if i+1==df.shape[0] or str(df.loc[i+1, col])== 'nan':
                        # 值为0，在最后一行或者下一行是nan，则说明0值是由 nan/非零值 算出来的，这个时候应该把0改成1
                        df.loc[i,col] = 1
                elif str(df.loc[i, col]) == 'nan':
                    df.loc[i,col] = 1
    return df

for symbol in sorted(set(finRatio.index)):
    df = finRatio.loc[symbol,:]
    if df.shape==(17,):
        # 只有一行，不用补了
        continue
    # finRatio.loc[symbol,:] = padding_values(df)
    # padding_cols = ['AQI', 'DSIR','DEPI', 'GMI', 'IG', 'LEV', 'RG', 'SG', 'SGEE']
    padding_cols = [5,8,9,10,11,12,14,15,16]
    # df.reset_index(drop=True, inplace=True)

    if df.iloc[:,5:].isnull().sum().sum()/(df.shape[0] * (df.shape[1]-5)) < 0.1:
        # 空值如果超过了 10%，就不用补了，之后直接把这家公司筛了
        for col in padding_cols:
            for i in range(df.shape[0]):
                if df.iloc[i, col]==0:
                    if i+1== df.shape[0] or str(df.iloc[i+1, col])== 'nan':
                        # 值为0，在最后一行或者下一行是nan，则说明0值是由 nan/非零值 算出来的，这个时候应该把0改成1
                        df.iloc[i,col] = 1
                elif str(df.iloc[i, col]) == 'nan':
                    df.iloc[i,col] = 1
finRatio.to_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_padded.xlsx',encoding='gb18030')

第二阶段筛选：在第一轮筛选过后，补齐了一些值，但还是有一些公司因为缺失值太多了，不适合补齐。第二轮筛选主要达到最后的文件中没有空值，包括去除一些金融公司和只有一年数据的公司

In [201]:
finRatio = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_padded.xlsx',encoding='gb18030')
# 把2002年的数据删了，因为2002年的数据只是用来后面计算Pi的。
finRatio.drop(finRatio[finRatio.EndDate==2002].index, axis=0, inplace=True)
finRatio = finRatio.set_index(['Symbol'])

n_nonfin_corp = set()
# n_no_weigui = set()
n_only_one_year = set()
n_no_missing_data = set()
drop_firms = []
print('初始公司有%s家' % len(set(finRatio.index)))

for firm in sorted(set(finRatio.index)):
    df = finRatio.loc[firm,:]
    if df.shape==(17,):
        # 只有一年的数据，也用不了
        n_only_one_year.add(firm)
        drop_firms.append(firm)
    else:
        ind = df.Industry.tolist() if type(df.Industry) is not str else df.Industry
        if '金融' not in ind:
            n_nonfin_corp.add(firm)
            if df.isnull().sum().sum()==0:
                # 判断指标缺失的公司
                n_no_missing_data.add(firm)
            else:
                drop_firms.append(firm)
        else:
            drop_firms.append(firm)

finRatio = finRatio.drop(drop_firms, axis=0)

print('共有%d家公司只有一年数据' % len(n_only_one_year))
print('共有%d家公司不是金融公司' % len(n_nonfin_corp))
# print('共有%d家公司没有财务报表欺诈' % len(n_no_weigui))
print('共有%d家公司没有缺失指标' % len(n_no_missing_data))
finRatio.to_excel('../data/会计信息质量-财务指标/all_corp_fin_data.xlsx',encoding='gb18030')

初始公司有4500家
共有482家公司只有一年数据
共有3899家公司不是金融公司
共有3855家公司没有缺失指标


计算JMIS2018另外72个指标\
Ti (i = 1, 2, …, 12) is the averaged Ri of five largest firms (in terms of sales) in the same industry and year\
Ci (i = 1, 2, …, 12) is the averaged Ri of five similar firms (in terms of sales) in the same industry and year\
Pi (i = 1, 2, …, 12) is the same ratio of Ri in the previous year.

In [202]:
# 先计算 Industry-collaboration contextual features
finRatio = pd.read_excel('../data/会计信息质量-财务指标/all_corp_fin_data.xlsx',encoding='gb18030')
group_by_ind_year = finRatio.set_index(['Industry','EndDate'])
# group_by_symbol_year 用来计算Ti，会涉及t-1年的数据，如果要算公司第一年的R1/P1，就会需要t-1年的数据，如2003年的R1/P1需要2002年的数据，而2002年的数据all_corp_fin_data里面没有，但AIQ_LCFinIndexY_padded里面有
group_by_symbol_year = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_padded.xlsx',encoding='gb18030').set_index(['Symbol','EndDate'])

base_fin_name = ['AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG','LEV', 'OPM', 'RG', 'SG', 'SGEE']
def generate_interval(index, total_size):
    if total_size < 8:
        return [i for i in range(total_size)]
    elif index >= 3 and index + 2 < total_size:
        return [index - 3, index - 2, index-1, index+1, index+2]
    elif index + 2 >= total_size:
        return [index - 5, index - 4, index - 3, index-2, index-1]
    else: #  index < 3
        return [index + 1, index + 2, index + 3, index+4, index+5]

for i in range(finRatio.shape[0]):
    stkcd, year = finRatio.loc[i, ['Symbol','EndDate']]
    ind = group_by_symbol_year.loc[(stkcd, year), 'Industry']
    # 按 sales 排序
    batch_df = group_by_ind_year.loc[(ind, year), :].copy().sort_values(by=['OperatingRevenue'], ascending=False)
    T = np.mean(
            # ['AQI', 'AT', 'CFED', 'DSIR', 'DEPI', 'GMI', 'IG','LEV', 'OPM', 'RG', 'SG', 'SGEE']
            batch_df.iloc[:5, [4,5,6,7,8,9,10,11,12,13,14,15]], axis=0
        )
    
    index_in_batch = batch_df.Symbol.tolist().index(stkcd)
    C = np.mean(
        batch_df.iloc[generate_interval(index_in_batch, batch_df.shape[0]),[4,5,6,7,8,9,10,11,12,13,14,15]], axis = 0
        )
    
    for j in range(len(base_fin_name)):
        ratio_name = base_fin_name[j]
        finRatio.loc[i, 'R%d-T%d' % (j+1,j+1) ] = finRatio.loc[i, ratio_name] - T[j]
        finRatio.loc[i, 'R%d/T%d' % (j+1,j+1) ] = finRatio.loc[i, ratio_name] / T[j]

    for j in range(len(base_fin_name)):
        ratio_name = base_fin_name[j]
        finRatio.loc[i, 'R%d-C%d' % (j+1,j+1) ] = finRatio.loc[i, ratio_name] - C[j]
        finRatio.loc[i, 'R%d/C%d' % (j+1,j+1) ] = finRatio.loc[i, ratio_name] / C[j]

    
    for j in range(len(base_fin_name)):
        try:
            # 2002年的数据用来计算2003年的R%d-P%d，算完以后删除2002年
            ratio_name = base_fin_name[j]
            finRatio.loc[i, 'R%d-P%d' % (j+1,j+1) ] = finRatio.loc[i, ratio_name] - group_by_symbol_year.loc[(stkcd, year-1), ratio_name]
            finRatio.loc[i, 'R%d/P%d' % (j+1,j+1) ] = finRatio.loc[i, ratio_name] / group_by_symbol_year.loc[(stkcd, year-1), ratio_name]
        except:
            # 说明没有t-1年的数据，暂时空着
            continue

finRatio.to_excel('../data/会计信息质量-财务指标/findata_84r.xlsx',encoding='gb18030',index=None)

/Users/luqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in double_scalars


有些公司-年份的72个指标中还是存在一些空值，需要补齐一下

In [203]:
finRatio = pd.read_excel('../data/会计信息质量-财务指标/findata_84r.xlsx',encoding='gb18030')
for i in range(12):
    # 识别 null 值，并用0或1补上
    finRatio['R%d-C%d' % (i+1, i+1)][finRatio['R%d-C%d' % (i+1, i+1)].isnull()==True] = 0
    finRatio['R%d/C%d' % (i+1, i+1)][finRatio['R%d/C%d' % (i+1, i+1)].isnull()==True] = 1

    finRatio['R%d-P%d' % (i+1, i+1)][finRatio['R%d-P%d' % (i+1, i+1)].isnull()==True] = 0
    finRatio['R%d/P%d' % (i+1, i+1)][finRatio['R%d/P%d' % (i+1, i+1)].isnull()==True] = 1

    # 识别 inf，并用0或1补上
    finRatio['R%d-C%d' % (i+1, i+1)][finRatio['R%d-C%d' % (i+1, i+1)]==np.inf] = 0
    finRatio['R%d/C%d' % (i+1, i+1)][finRatio['R%d/C%d' % (i+1, i+1)]==np.inf] = 1

    finRatio['R%d-P%d' % (i+1, i+1)][finRatio['R%d-P%d' % (i+1, i+1)]==np.inf] = 0
    finRatio['R%d/P%d' % (i+1, i+1)][finRatio['R%d/P%d' % (i+1, i+1)]==np.inf] = 1
    
finRatio.to_excel('../data/会计信息质量-财务指标/findata_84r.xlsx',encoding='gb18030',index=None)

/Users/luqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/luqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/luqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/luqi/op

最后的最后，以公司为单位做标准化

In [204]:
finRatio = pd.read_excel('../data/会计信息质量-财务指标/findata_84r.xlsx',encoding='gb18030')

# temp = pd.DataFrame(columns=finRatio.columns,dtype=object)
# temp.to_csv('会计信息质量-财务指标/findata_normlized.csv', encoding='gb18030',index=None)

finRatio.set_index('Symbol', inplace=True)
BN = torch.nn.BatchNorm1d(84, eps=1e-4)
for i in sorted(set(finRatio.index)):
    df = finRatio.loc[i,:]
    temp = df.copy(deep=True)
    normed_value = BN(torch.from_numpy(df.iloc[:,5:].values).float()).data.numpy()
    temp.iloc[:,5:] = normed_value
    finRatio.loc[i,:] = temp.values
finRatio.to_excel('../data/会计信息质量-财务指标/findata_normlized.xlsx', encoding='gb18030')

#### 处理 non-financial ratios,处理完以后放在各自的文件夹里，并且生成一个总的文件 nonfin_data.csv

<u>**董事会特征Boardroom Characteristics**</u> (4 ratios)\
1.外部董事比例(The proportion of outside directors)\
2.董事会规模(The size of the board)\
3.董事会会议次数(the number of board meetings)\
4.董事会主席任期(the tenure of the chairman)

<u>**所有人结构Ownership Structure**</u> (6 ratios)\
1.国家持股比例(The proportion of shares held by the state)\
2.法人持股比例(the proportion of shares held by legal entities)\
3.个人持股比例(the proportion of shares held by individuals)\
4.是否存在外资股东(the existence of foreign stockholders)\
5.单一第一大股东的持股比例(the proportion of shares held by the single largest stockholder)\
6.第二至第十大股东持股比例(the concentration of ownership in the hands of the second to tenth largest stockholders)

<u>**审计Audit**</u> (6 ratios)\
审计质量(Auditor Quality)

In [ ]:
# 董事会规模、外部董事比例
ratio_out_directors = pd.read_excel('../data/非财务指标/高管人数持股及薪酬情况表/CG_ManagerShareSalary_raw.xlsx')
ratio_out_directors.drop([0,1],axis=0,inplace=True)

ratio_out_directors.Enddate = [int(t.split('-')[0]) for t in ratio_out_directors.Enddate]
ratio_out_directors.drop(['StatisticalCaliber'],axis=1,inplace=True)
ratio_out_directors.drop_duplicates(subset=['Symbol','Enddate'],keep='last',inplace=True)
ratio_out_directors.to_csv('../data/非财务指标/高管人数持股及薪酬情况表/CG_ManagerShareSalary.csv', index=None)

In [ ]:
# 董事会会议次数
'''
A0101b: 董事会会议次数
A0201b: 监事会会议次数
A0301b: 股东大会召开次数
'''
board_meet = pd.read_excel('../data/非财务指标/三会基本信息文件/CG_Agm_raw.xlsx')
board_meet.drop([0,1],axis=0,inplace=True)
board_meet.Reptdt = [int(t.split('-')[0]) for t in board_meet.Reptdt]
board_meet.to_csv('../data/非财务指标/三会基本信息文件/CG_Agm.csv',index=None)

In [ ]:
# 董事会主席任期
chairman_tenure = pd.read_excel('../data/非财务指标/董监高任职情况表/TMT_POSITION_raw.xlsx')
chairman_tenure.drop([0,1],axis=0,inplace=True)
chairman_tenure.Reptdt = [int(t.split('-')[0]) for t in chairman_tenure.Reptdt]
chairman_tenure.drop_duplicates(subset=['Stkcd','Reptdt'],keep='last',inplace=True)
chairman_tenure.to_csv('../data/非财务指标/董监高任职情况表/TMT_POSITION.csv',index=None,encoding='gb18030')

In [ ]:
# 国家持股、法人持股、个人持股、是否存在外资股东
sharehold = pd.read_excel('../data/非财务指标/股本结构文件/CG_Capchg_raw.xlsx')
sharehold.drop([0,1],axis=0,inplace=True)
sharehold.Reptdt = [int(t.split('-')[0]) for t in sharehold.Reptdt]
sharehold['nation_rate'] = [nat/total for total, nat in zip(sharehold.Nshrttl, sharehold.Nshrstt)]
sharehold['legal_rate'] = [(in_legal+out_legal)/total for total, in_legal, out_legal in zip(sharehold.Nshrttl, sharehold.Nshrlpd, sharehold.Nshrlpf)]
sharehold['personal_rate'] = [ind/total for total, ind in zip(sharehold.Nshrttl, sharehold.Nshrsms)]
sharehold['is_abroad_exist'] = [1 if out_legal>0 else 0 for out_legal in sharehold.Nshrlpf]
sharehold.to_csv('../data/非财务指标/股本结构文件/CG_Capchg.csv',index=None)

In [ ]:
# 第一大股东持股比例、第二至第十大股东持股比例
'''
Shrcr1: 第一大股东持股比例
Shrcr4: 前十大股东持股比例
'''
share_hold_rate = pd.read_excel('../data/非财务指标/十大股东股权集中文件/HLD_CR_raw.xlsx')
share_hold_rate.drop([0,1],axis=0,inplace=True)
share_hold_rate.Reptdt = [int(t.split('-')[0]) for t in share_hold_rate.Reptdt]
share_hold_rate.drop_duplicates(subset=['Stkcd','Reptdt'],keep='last',inplace=True)
share_hold_rate['1st_rate'] = share_hold_rate.Shrcr1
share_hold_rate['2nd_10th_rate'] = [j-i for i,j in zip(share_hold_rate.Shrcr1, share_hold_rate.Shrcr4)]

share_hold_rate.to_csv('../data/非财务指标/十大股东股权集中文件/HLD_CR.csv',index=None)

In [ ]:
# 审计质量
auditor_info = pd.read_excel('../data/非财务指标/中国百强会计师事务所排名表/AR_TOP100AFRANK.xlsx')
auditor_info.drop([0,1],axis=0,inplace=True)
auditor_info.set_index(['InstitutionID'],inplace=True)

corp_audit_info = pd.read_excel('../data/非财务指标/上市公司审计机构列表/AR_LISTCOMPAUDIT_raw.xlsx')
corp_audit_info.drop([0,1],axis=0,inplace=True)
corp_audit_info.EndDate = [int(t.split('-')[0]) for t in corp_audit_info.EndDate]

for i in corp_audit_info.index:
    Ins_id = corp_audit_info.loc[i,'InstitutionID']
    Ins_name = corp_audit_info.loc[i,'AccountingFirmName']
    if Ins_id in set(auditor_info.index):
        if corp_audit_info.loc[i,'TerritoryForeignIdentity']==1:
            score_list = auditor_info.loc[Ins_id, 'CompositeScore']
            corp_audit_info.loc[i,'AuditQuality'] = score_list.mean() if type(score_list)==pd.core.series.Series else score_list

corp_audit_info.drop_duplicates(subset=['Symbol','EndDate'],keep='first',inplace=True)
corp_audit_info[corp_audit_info.TerritoryForeignIdentity==1].to_csv('../data/非财务指标/上市公司审计机构列表/AR_LISTCOMPAUDIT.csv',encoding='gb18030',index=None)

In [ ]:
'''
各个非财务指标处理结束，
把所有 nonfinancial 指标的表拼接起来
'''
ratio_out_directors = pd.read_csv('../data/非财务指标/高管人数持股及薪酬情况表/CG_ManagerShareSalary.csv', encoding='gb18030')
board_meet = pd.read_csv('../data/非财务指标/三会基本信息文件/CG_Agm.csv', encoding='gb18030')
chairman_tenure = pd.read_csv('../data/非财务指标/董监高任职情况表/TMT_POSITION.csv', encoding='gb18030')
sharehold = pd.read_csv('../data/非财务指标/股本结构文件/CG_Capchg.csv', encoding='gb18030')
share_hold_rate = pd.read_csv('../data/非财务指标/十大股东股权集中文件/HLD_CR.csv', encoding='gb18030')
corp_audit_info = pd.read_csv('../data/非财务指标/上市公司审计机构列表/AR_LISTCOMPAUDIT.csv',encoding='gb18030')

ratio_out_directors.set_index(['Symbol','Enddate'],inplace=True)
board_meet.set_index(['Stkcd','Reptdt'],inplace=True)
chairman_tenure.set_index(['Stkcd','Reptdt'], inplace=True)
sharehold.set_index(['Stkcd','Reptdt'],inplace=True)
share_hold_rate.set_index(['Stkcd','Reptdt'],inplace=True)
corp_audit_info.set_index(['Symbol','EndDate'],inplace=True)

In [ ]:
nonfin_data = pd.concat([ratio_out_directors, 
            board_meet.A0101b, 
            chairman_tenure.Tenure, 
            sharehold[['nation_rate', 'legal_rate', 'personal_rate','is_abroad_exist']], 
            share_hold_rate[['1st_rate', '2nd_10th_rate']],
            corp_audit_info.AuditQuality,
        ],axis=1,ignore_index=False)

nonfin_data.reset_index(drop=False,inplace=True)
nonfin_data.rename(columns={'level_0':'Symbol','level_1':'year', 'A0101b':'board_meeting_num'},inplace=True)
nonfin_data.to_csv('../data/非财务指标/nonfin_data_raw.csv', encoding='gb18030',index=None)

#### nonfin_data文件构造完成，接下来要删除一些0值过多的行，补齐 nonfin_data 的空值

In [ ]:
nonfin_data = pd.read_csv('../data/非财务指标/nonfin_data_raw.csv')

# 22年空值有很多，把22年的行删了，2000、2001、2002年也一样
nonfin_data.set_index(['year'],inplace=True)
nonfin_data.drop([2000, 2001, 2002, 2022],inplace=True, axis=0)
nonfin_data.reset_index(drop=False,inplace=True)
nonfin_data.to_csv('../data/非财务指标/nonfin_data.csv',encoding='gb18030',index=None)

In [59]:
nonfin_data = pd.read_csv('../data/非财务指标/nonfin_data.csv',encoding='gb18030')

drop_index = []
# 先把一些空值特别多(>9)的行删了
for i in nonfin_data.index:
    if nonfin_data.loc[i,:].isnull().sum() >= 8:
        drop_index.append(i)
print('有%d行的指标几乎全是nan' % len(drop_index))
nonfin_data.drop(drop_index, inplace=True, axis=0)
# nonfin_data = nonfin_data.set_index(['Symbol'])


有11499行的指标几乎全是nan


In [130]:
n_only_one_year = []

def padding_by_column(col):
    for i in range(len(col)-2, -1, -1):
        # 从倒数第二个开始，往前倒
        if str(col[i]) == 'nan':
            col[i] = col[i+1]
    for i in range(1, len(col)):
        # 再从正数第二个开始，往后倒
        if str(col[i]) == 'nan':
            col[i] = col[i-1]
    return col        

nonfin_by_firm = nonfin_data.set_index(['Symbol'],inplace=False)
for firm in set(nonfin_by_firm.index):
    df = nonfin_by_firm.loc[firm, :]
    if df.shape == (12,):
        # 只有一行，不满足时间序列要求，删掉
        n_only_one_year.append(firm)
        continue
    # 先补 tenure
    tenure = df.Tenure.copy().tolist()
    for i in range(len(tenure)-2, -1, -1):
        # 从倒数第二个开始，往前倒
        if str(tenure[i]) == 'nan':
            tenure[i] = tenure[i+1] - 12 if tenure[i+1] > 12 else np.nan
    for i in range(1, len(tenure)):
        # 再从正数第二个开始，往后倒
        if str(tenure[i]) == 'nan':
            tenure[i] = tenure[i-1] + 12
    nonfin_by_firm.loc[firm, 'Tenure'] = tenure
    # 再补别的
    nonfin_by_firm.loc[firm, 'DirectorNumber'] = padding_by_column(df.DirectorNumber.copy().tolist())
    nonfin_by_firm.loc[firm, 'IndependentDirectorNumber'] = padding_by_column(df.IndependentDirectorNumber.copy().tolist())
    nonfin_by_firm.loc[firm, 'board_meeting_num'] = padding_by_column(df.board_meeting_num.copy().tolist())
    nonfin_by_firm.loc[firm, 'nation_rate'] = padding_by_column(df.nation_rate.copy().tolist())
    nonfin_by_firm.loc[firm, 'legal_rate'] = padding_by_column(df.legal_rate.copy().tolist())
    nonfin_by_firm.loc[firm, 'personal_rate'] = padding_by_column(df.personal_rate.copy().tolist())
    nonfin_by_firm.loc[firm, '1st_rate'] = padding_by_column(df["1st_rate"].copy().tolist())
    nonfin_by_firm.loc[firm, '2nd_10th_rate'] = padding_by_column(df["2nd_10th_rate"].copy().tolist())
    nonfin_by_firm.loc[firm, 'AuditQuality'] = padding_by_column(df.AuditQuality.copy().tolist())

print('有%d家公司只有一年数据，删掉'%len(n_only_one_year))
nonfin_by_firm.drop(n_only_one_year, inplace=True, axis=0)
nonfin = nonfin_by_firm.reset_index(drop=False,inplace=False)
'''
补完以后，还有一些地方有空值，比如Tenure，AuditQuality，是因为公司所有年份的值都是空的，就不能根据上下两行的数据来补
因此将有空值的行全删了，保证整个表格没有一处地方有空值
'''
drop_index = []
for i in nonfin.index:
    if nonfin.loc[i,:].isnull().sum() >0 :
        drop_index.append(i)
print('补充完之后，还有%d行数据有 nan 值，这些值因为上下文也是空的所以无法补充' % len(drop_index))
nonfin.drop(drop_index, inplace=True, axis = 0)

nonfin.to_csv('../data/非财务指标/nonfin_data_v2.csv',encoding='gb18030',index=None)


有541家公司只有一年数据，删掉
补充完之后，还有85行数据有 nan 值，这些值因为上下文也是空的所以无法补充


##### nonfinancial ratios 的处理暂时结束

In [41]:
nonfin = pd.read_csv('../data/非财务指标/nonfin_data_v2.csv',encoding='gb18030')

# temp = pd.DataFrame(columns=finRatio.columns,dtype=object)
# temp.to_csv('会计信息质量-财务指标/findata_normlized.csv', encoding='gb18030',index=None)

nonfin.set_index('Symbol', inplace=True)
BN = torch.nn.BatchNorm1d(11, eps=1e-4)
normed_value = BN(torch.from_numpy(nonfin.iloc[:,1:].values).float()).data.numpy()

nonfin.loc[:,1:] = normed_value
'''for i in sorted(set(finRatio.index)):
    df = nonfin.loc[i,:]
    temp = df.copy(deep=True)
    normed_value = BN(torch.from_numpy(df.iloc[:,5:].values).float()).data.numpy()
    temp.iloc[:,5:] = normed_value
    finRatio.loc[i,:] = temp.values'''
nonfin.to_csv('../data/非财务指标/nonfin_normlized.csv', encoding='gb18030')

In [40]:
nonfin

,Symbol,year,DirectorNumber,IndependentDirectorNumber,board_meeting_num,Tenure,nation_rate,legal_rate,personal_rate,is_abroad_exist,1st_rate,2nd_10th_rate,AuditQuality
0,1,2003,14.0,3.0,13.0,3.0,0.064441,0.000000,0.000000,0.0,7.0838,18.9276,59.944286
1,1,2004,13.0,5.0,15.0,1.0,0.000882,0.274816,0.000000,1.0,17.8898,8.8375,59.944286
2,1,2005,14.0,4.0,10.0,6.0,0.000882,0.274816,0.000000,1.0,17.8898,9.3922,59.944286
3,1,2006,13.0,4.0,12.0,18.0,0.000882,0.274816,0.000000,1.0,17.8898,11.8932,59.944286
4,1,2007,14.0,4.0,16.0,30.0,0.002017,0.231715,0.000006,1.0,16.6963,14.3021,1019.076154
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47958,900957,2017,9.0,3.0,7.0,43.0,0.000000,0.472779,0.000000,0.0,29.6189,18.8200,728.642500
47959,900957,2018,9.0,3.0,4.0,55.0,0.000000,0.472779,0.000000,0.0,29.6189,18.4098,728.642500
47960,900957,2019,9.0,3.0,4.0,67.0,0.000000,0.472779,0.000000,0.0,29.6189,18.5604,728.642500
47961,900957,2020,11.0,5.0,5.0,79.0,0.000000,0.472779,0.000000,0.0,29.6189,18.6904,728.642500


#### 管理层讨论与分析数据处理：2001-2021年
数据来源于闲鱼，是从中国研究数据服务平台上搞下来的，[人大数字图书馆](https://libproxy.ruc.edu.cn/ermsClient/eresourceInfo.do?rid=121)也有这个数据库的链接，只不过没有开通管理层讨论与分析的权限\
年报文件夹里的其他文件都是当时用来爬年报和解析年报用的，有了管理层讨论与分析文件夹里面的数据，这些文件就用不到了


In [4]:
# 读取mda原始文件和中文停用词
mda = pd.read_excel('../data/年报/管理层讨论与分析/管理层讨论与分析_raw.xlsx')
mda = mda.drop(index=[0],axis=0)
drop_half_year = []
for i in mda.index:
    if mda.loc[i,'Anatime'].split('-')[1] == '06':
        # 去除半年度的报告
        drop_half_year.append(i)
mda.drop(index=drop_half_year, axis=0, inplace=True)
mda.reset_index(drop=True, inplace=True)

stop_words = []
# 读取停用词表
with open('../data/年报/chineseStopWords.txt','r') as f:
    for line in f.readlines():
        stop_words.append(line.replace('\n',''))


去除停用词，数字和字符 (Bert的话可以去除停用词，也可以不去)，但由于篇幅的原因，还是尽量去掉，减轻计算量

In [222]:
# 文本预处理
new_BusDA = []
for text in mda.BusDA:
    t1 = re.sub(r'\n','',text)
    # 1,699,196,509 元 -> 1699196509 元
    t2 = re.sub(r',','',t1)
    # 去除数字
    t3 = re.sub(r'[0-9]+','',t2)
    t4 = ''
    # 去除停用词
    for word in jieba.cut(t3):
        if word not in stop_words:
            t4 += word
            t4 += " "
    new_BusDA.append(t4)
    
mda.BusDA = new_BusDA
# 要运行 96 分钟
# csv 文件太大了，生成了不方便打开
mda.to_excel('../data/年报/管理层讨论与分析/mda_no_number.xlsx',encoding='gb18030',index=None)


In [5]:
mda = pd.read_excel('../data/年报/管理层讨论与分析/mda_no_number.xlsx', encoding='gb18030')

In [6]:
for text in mda.BusDA[:]:
    print(text)
    break

年度 本行 主营业务 本行 主营业务 经营范围 中国人民银行 批准 各项 商业银行 业务 包括 人民币 外币 存贷款 国际 国内 结算 票据 贴现 外汇 买卖 提供 担保 信用证 服务 提供 保管箱 服务 报告 期内 经营 情况 本行 经营 班子 贯彻落实 董事会 提出 各项 业务 发展 计划 措施 规范 经营 立行 资产 质量 立行 人才 素质 立行 经济效益 立行 经营 方针 领导 组织 全行 员工 紧紧围绕 一手 抓 市场 一手 抓 质量 这两项 中心 工作 团结 进取 努力 开拓 使 本行 日益 激烈 市场竞争 环境 各项 业务 稳步发展 报告 期内 本行 经营 情况 内部 管理 进一步 强化 报告 期内 本行 引入 独立 董事 制度 进一步 完善 公司法人 治理 结构 健全 内控 机制 强化 风险 控制 本行 内部 资源整合 考核 力度 进一步提高 全行 管理水平 工作效率 报告 期末 本行 监管 指标 符合 中国人民银行 资产 规模 持续 报告 期末 本行 总资产 亿元   年初 增加 亿元 增幅 ％ 本外币 贷款 余额 亿元 年初 增加 亿元   增幅 ％ 存款 增长 报告 期末 本行 各项 存款 余额 亿元   年初 增加   亿元 增幅 ％ 本外币 各项 储蓄存款 余额 亿元   年初 增加 亿元 增幅 ％ 资产 质量 改善 报告 期末 逾 呆 口径 计算 本行 不良贷款 余额   亿元 年初 减少 亿元 不良贷款 率为 ％ 年初 ％ 下降 百分点 各项 业务 发展 报告 期内 本行 存 贷款 大幅 增长 业务 快 发展 全年 新增 发卡 万张 国际 结算 量 亿美元 年初 增长 ％ 全年 各类 中间业务 创造 非 利息收入 亿元 税前 利润 有所增加 报告 期内 本行 税前 利润 亿元 年初 增加 亿元 增幅 ％ 机构 拓展 新 进展 报告 期内 本行 新开设 天津 分行 济南 分行 筹备 青岛 分行 贵阳 分行 成都 分行 全年 本行 营业网点 年初   家 增加 年末 家 科技 建设 较大 进展 报告 期内 采用 前置 构架 全行   一本 帐 先进 特征 新一代 综合 客户服务 系统 天津 珠海   重庆 济南 分支机构 按计划 投入 运行 中国人民银行 年末 全国 商业银行 信贷 报表 资料 统计 报告 期末 全国 家 同类 商业银行 本

中文Bert代码调用，参考[HuggingFace: bert-based-chinese](https://huggingface.co/bert-base-chinese)，[Bert如何解决长文本问题-知乎](https://www.zhihu.com/question/327450789)，[bert训练数据预处理方法](https://blog.csdn.net/weixin_43643246/article/details/104389624)

In [27]:
from transformers import BertConfig, BertModel, AutoTokenizer, AutoModelForMaskedLM, AutoModel
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config


In [2]:
configuration

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
text = mda.BusDA[5]
text

'报告 期内 经营 情况 讨论 分析 主营业务 本行 主营业务 经营范围 中国 银行业 监督管理 委员会 批准 各项 商业银行 业务 包括 办理 人民币 存 贷 结算 汇兑 业务 人民币 票据 承兑 贴现 各项 信托 业务 中国 银行业 监督管理 委员会 批准 发行 买卖 人民币 有价证券 外汇 存款 汇款 境内 境外 借款 境内 境外 发行 代理 发行 外币 有价证券 贸易 非贸易 结算 外币 票据 承兑 贴现 外汇 放款 代客 买卖 外汇 外币 有价证券 自营 外汇 买卖 资信 调查 咨询 见证 业务 中国 银行业 监督管理 委员会 批准 业务 本行 国家 全国性 经营 商业银行 本行 战略性 经营 网络 中国 发达 地区 珠江三角洲 环渤海地区 长江三角洲 本行 正 发展 中国 西部 城市 网络 本行 净 利息收入 增长 % 亿元 人民币 得益于 存款 业务 支持 基础 零售 贷款 业务 产出 利息收入 达 亿元 人民币 贷款 票据 贴现 业务 带来 利息收入 本行 利息收入 来源 本行 在结构上 一般性 贷款 业务 调整 一般性 贷款 带来 利息收入 占 总 利息收入 % 本行 资产 混合 配置管理 方针 本行 贴现 余额 下降 % 一般性 贷款 增加 % 零售 贷款 增幅 达 % 信用卡 贷款 款项 房贷 增长 % % 利率 提高 贷款 结构调整 房贷 业务 迅猛 增长 造就 本行 截止 年月日 一般性 贷款 利息收入 增长 % 实施 更为 资金 成本 管理 利息支出 增长 % 亿元 人民币 年度 本行 资产负债 管理 调整 生息 资产负债 结构 提高 高 生息 资产 比例 减少 高 生息 负债 加上 紧密 动态 利差 率 管理 资金运用 管理 生息 资产 利差 率 上年 百分点 % 因素 均 资金运用 效率 提高 本行 净 利息收入 增长 银行业务 贷款 本行 带来 % 贷款 利息收入 本行 供应链 金融 统率 业务 发展 转型 贸易 融资 专业银行 广受 赞誉 本行 借助 国际 先进 贸易 融资 成熟技术 研发 实施 自偿性 贸易 融资 授信 评级 制度 创新 完善 全程 供应链 金融 产品开发 投产 贸易 融资 操作系统 建立 贸易 融资 产品 经理 团队 深化 中外运 中储 中远 物流 中华 商务网 战略伙伴 合作 初步 构建 供应链 金融 — 物

In [29]:
cur_sentence = ''
# bert支持的单个最长的文本是512
MAX_SUBSEN_LEN = 500

inputs = []
for s in text.split(' '):
    if len(cur_sentence + s) <= MAX_SUBSEN_LEN:
        cur_sentence += s
    else:
        inputs.append(cur_sentence)
        cur_sentence = s
# 手动加上最后一段
inputs.append(cur_sentence)
inputs
# 两位数字占一个字符，一个汉字占一个字符
# 中文单字分隔，英文数字分词

['年度本行主营业务本行主营业务经营范围中国人民银行批准各项商业银行业务包括人民币外币存贷款国际国内结算票据贴现外汇买卖提供担保信用证服务提供保管箱服务报告期内经营情况本行经营班子贯彻落实董事会提出各项业务发展计划措施规范经营立行资产质量立行人才素质立行经济效益立行经营方针领导组织全行员工紧紧围绕一手抓市场一手抓质量这两项中心工作团结进取努力开拓使本行日益激烈市场竞争环境各项业务稳步发展报告期内本行经营情况内部管理进一步强化报告期内本行引入独立董事制度进一步完善公司法人治理结构健全内控机制强化风险控制本行内部资源整合考核力度进一步提高全行管理水平工作效率报告期末本行监管指标符合中国人民银行资产规模持续报告期末本行总资产亿元年初增加亿元增幅％本外币贷款余额亿元年初增加亿元增幅％存款增长报告期末本行各项存款余额亿元年初增加亿元增幅％本外币各项储蓄存款余额亿元年初增加亿元增幅％资产质量改善报告期末逾呆口径计算本行不良贷款余额亿元年初减少亿元不良贷款率为％年初％下降百分点各项业务发展报告期内本行存贷款大幅增长业务快发展全年新增发卡万张国际结算量亿美元年初增长％全年各类中间业务创造非利息收入亿元税前',
 '利润有所增加报告期内本行税前利润亿元年初增加亿元增幅％机构拓展新进展报告期内本行新开设天津分行济南分行筹备青岛分行贵阳分行成都分行全年本行营业网点年初家增加年末家科技建设较大进展报告期内采用前置构架全行一本帐先进特征新一代综合客户服务系统天津珠海重庆济南分支机构按计划投入运行中国人民银行年末全国商业银行信贷报表资料统计报告期末全国家同类商业银行本行各项存款占市场份额居第位增长幅度居第位各项贷款占市场份额居第位增长幅度居第位资产利润率居第位报告期内占主营业务收入％业务经营活动情况报告期内占本行主营业务收入％业务利息收入本行利息收入元利息支出元净利息收入元本行控股公司参股经营情况业绩报告期内本行无新增控股公司投资股权投资中国人民银行本行控股子公司元盛股权投资办理脱钩清理工作见会计报表附注会计报表附注十四年度经营计划本行经营计划四个立行经营方针基础进一步完善市场开拓体系风险控制体系支援保障体系推进体制创新战略业务特色战略科技先导战略人才培养战略面向市场信息把握风险控制成本规模跨越式增长质量高水平控制效益综合性提高提升本行综合竞争能力崭新面貌应对WTO中外金融业竞争着重做好工作整

In [32]:
tokens = []
embs = []
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

for section in inputs:
    encoded_input = tokenizer(section, return_tensors='pt',padding=True).to(DEVICE)
    # tokens.append(encoded_input['input_ids'])
    output = model(**encoded_input, return_dict=True)
    # 提取出每一个句子的 CLS 向量
    # print(output.last_hidden_state[:,0,:].shape)
    embs.append(output.last_hidden_state[:,0,:].squeeze().tolist())
# embs = torch.tensor(embs)

In [33]:
embs = torch.tensor(embs)
print(embs.shape)

torch.Size([3, 768])


In [91]:
empty = torch.tensor([])
torch.concat([empty, pooling_layer(embs.T).T, pooling_layer(embs.T).T], dim=0)

tensor([[-8.3204, -8.3937, -8.3671,  ..., -7.3723, -7.7074, -7.6464],
        [-8.3204, -8.3937, -8.3671,  ..., -7.3723, -7.7074, -7.6464]])

In [85]:
pooling_layer = torch.nn.MaxPool1d(kernel_size=embs.shape[0])
pooling_layer(embs.T).T

torch.Size([1, 21128])

In [13]:
# text = ['本行总资产达到82亿元。']
# return_tensors: 'tf'-tensorflow, 'pt'-pytorch
encoded_input = tokenizer(text, return_tensors='pt',padding=True)
encoded_input['input_ids']
print(encoded_input['input_ids'].shape)

torch.Size([1, 1220])


In [101]:
encoded_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [3]:
mda = pd.read_excel('../data/年报/管理层讨论与分析/mda_no_number.xlsx')

In [99]:
mda.to_excel('../data/年报/管理层讨论与分析/mda_no_number.xls', encoding='gb18030', index=None)

In [102]:
output = model(**encoded_input)

In [103]:
encoded_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [175]:
# 提取出每一个句子的 CLS 向量
output.logits[:,0]

tensor([[-9.0493, -9.0590, -8.8648,  ..., -7.4138, -7.1695, -8.6114],
        [-8.5336, -8.5681, -8.3390,  ..., -7.4831, -7.0072, -7.9200],
        [-8.7777, -8.7546, -8.5135,  ..., -7.3962, -7.1147, -7.7257]],
       grad_fn=<SelectBackward0>)

In [178]:
# 把文本的不同段通过池化的方法合并
pooling_layer = torch.nn.MaxPool1d(kernel_size=2)
pooling_layer(output.logits[:,0].T).T

tensor([[-8.7914, -8.8136, -8.6019,  ..., -7.4484, -7.0883, -8.2657]],
       grad_fn=<PermuteBackward0>)

In [72]:
# 解决长文本的样例代码，来源于知乎 https://www.zhihu.com/question/327450789
class LongTextDataloader(object):
    def __init__(self, filename: str, max_sub_sentence_len: int, batch_size: int, 
                    shuffle=False):
        """
        长文本dataloader，初始化函数。

        Args:
            filename (str): 数据集文件
            max_sub_sentence_len (int): 每个子句最大的长度限制
            batch_size (int): 一次返回多少句子
            shuffle (bool): 是否打乱数据集
        """
        self.texts, self.labels = self.__read_file(filename)
        assert len(self.texts) == len(self.labels), '[ERROR] texts count not equal label count.'
        self.start = 0
        self.end = len(self.texts)
        self.batch_size = batch_size
        self.max_sub_sentence_len = max_sub_sentence_len
        self.visit_order = [i for i in range(self.end)]
        if shuffle:
            random.shuffle(self.visit_order)
    
    def __read_file(self, filename: str) -> tuple:
        """
        将本地数据集读到数据加载器中。

        Args:
            filename (str): 数据集文件名

        Returns:
            [tuple] -> 文本列表，标签列表
        """
        texts, labels = [], []
        with open(filename, 'r', encoding='utf8') as f:
            for line in f.readlines():
                label, text = line.strip().split('\t')
                texts.append(text)
                labels.append(label)
        return texts, labels

    def __split_long_text(self, text: str) -> list:
        """
        用于迭代器返回数据样本的时候将长文本切割为若干条。

        Args:
            text (str): 长文本, e.g. -> "我爱中国"
        
        Returns:
            [list] -> ["我爱", "中国"]（假设self.max_sub_sentence_len = 2）
        """
        sub_texts, start, length = [], 0, len(text)
        while start < length:
            sub_texts.append(text[start: start + self.max_sub_sentence_len])
            start += self.max_sub_sentence_len
        return sub_texts
    
    def __next__(self) -> dict:
        """
        迭代器，每次返回数据集中的一个样本，返回样本前会先将长文本切割为若干个短句子。

        Raises:
            StopIteration: [description]

        Returns:
            [dict] -> {
                'text': [sub_sentence 1, sub_sentence 2, ...],
                'label': 1
            }
        """
        if self.start < self.end:
            ret = self.start
            batch_end = ret + self.batch_size
            self.start += self.batch_size
            currents = self.visit_order[ret: batch_end]
            return {'text': [self.__split_long_text(self.texts[c]) for c in currents], 'label': [int(self.labels[c]) for c in currents]}
        else:
            self.start = 0
            raise StopIteration
    
    def __iter__(self):
        return self

    def __len__(self) -> int:
        return len(self.labels)

[GPT2 Chinese](https://huggingface.co/uer/gpt2-chinese-cluecorpussmall) 也可以做表示学习，但参数可能比 Bert 多。Bert是3亿参数量;而普通的GPT-2是15亿参数量。而 GPT-3没有中文版

In [201]:
from transformers import GPT2Tokenizer, GPT2Model, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('uer/gpt2-chinese-cluecorpussmall')
model = GPT2Model.from_pretrained('uer/gpt2-chinese-cluecorpussmall')

# text = "Replace me by any text you'd like."
text = '本行总资产达到82亿元。'
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(encoded_input['input_ids'].shape, output.shape)

Some weights of the model checkpoint at uer/gpt2-chinese-cluecorpussmall were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### 关系数据处理

In [14]:
# 将上市公司全称和股票代码对应起来:
stk_map_name_id = pd.DataFrame(columns=['FullName','Stkcd'],dtype=object)

name_id_dict = {}
corp_basic_info = pd.read_excel('../data/上市公司基本情况/STK_LISTEDCOINFOANL.xlsx',encoding='gb18030')
corp_basic_info.drop([0,1],inplace=True)
for i in corp_basic_info.index:
    key = corp_basic_info.loc[i,'FullName']
    value = corp_basic_info.loc[i,'Symbol']
    name_id_dict[key] = value

stk_map_name_id.FullName = name_id_dict.keys()
stk_map_name_id.Stkcd = name_id_dict.values()
# stk_map_name_id.to_csv('../data/stk_fullname.csv',encoding='gb18030',index=None)

##### 高管关系

In [24]:
# 初步处理
gaoguan = pd.read_excel('../data/高管关系/CG_Director_raw.xlsx',encoding='gb18030')
gaoguan.drop([0,1],axis=0,inplace=True)

# 把D0701b(现职任职开始日期)、D0702b(现职任职结束日期)删掉
gaoguan.drop(['D0701b', 'D0702b'], axis=1, inplace=True)
gaoguan = gaoguan.reset_index(drop=True)

# 修改列名
gaoguan.rename(columns={'D0101b': 'manager', 'PersonID':'manager_id', 'D0201b':'position', 'Reptdt': 'interval'},inplace=True)
gaoguan.interval = [t.split('-')[0] for t in gaoguan.interval]

# gaoguan.csv：所有上市公司，不只是train_test.csv里面的公司
gaoguan.to_csv('../data/高管关系/gaoguan.csv',encoding='gb18030',index=None)

生成两个字典：repeat_names 和 duel_people_stk_id，在后面的亲属、关联、股东关系中都会用到

In [5]:
gaoguan = pd.read_csv('../data/高管关系/gaoguan.csv',encoding='gb18030')
people_id = {}
repeat_names = set()

for i in gaoguan.index:
    name = gaoguan.loc[i,'manager']
    id = gaoguan.loc[i,'manager_id']
    if name in people_id.keys():
        if id not in people_id[name]:
            people_id[name].append(id)
            repeat_names.add(name)
    else:
        people_id[name] = [id]
print(len(people_id),len(repeat_names))

# 构造一个字典中的字典：name：stk：id，这个字典用于在扫描所有关系中的重名人，在人名后面加上-id
duel_people_stk_id = {}
for i in gaoguan.index:
    name = gaoguan.loc[i,'manager']
    stk = gaoguan.loc[i,'Stkcd']
    id = gaoguan.loc[i,'manager_id']
    if name in repeat_names:
        # 人名重复
        if name not in duel_people_stk_id:
            duel_people_stk_id[name] = {}
        #if stk not in people_stk_id[name]:
        duel_people_stk_id[name][stk] = id

114468 12083


In [26]:
# 高管关系人名配对
gaoguan = pd.read_csv('../data/高管关系/gaoguan.csv',encoding='gb18030')
gaoguan.manager = [name+'-'+str(duel_people_stk_id[name][stk]) if name in repeat_names else name for name, stk in zip(gaoguan.manager, gaoguan.Stkcd)]
gaoguan.to_csv('../data/高管关系/gaoguan_no_dup_person.csv',encoding='gb18030',index=None)

##### 亲属关系

In [2]:
qinshu = pd.read_excel('../data/股权及亲属关系/HLD_Shrrelchain.xlsx',encoding='gb18030')
qinshu.drop([0,1],inplace=True)
qinshu = qinshu.reset_index(drop=True)
print(len(set(qinshu.Stkcd)))

3838


In [3]:
# 要用 base 环境跑，用 anaconda3 跑会报错 pandas
# 把所有个体人名记下来，在收集股东关系的时候把人名往外扩
pattern = r'近*亲属|配偶的*|夫妻|[父母][子女亲]|[堂亲长胞]*[兄弟姐妹]+的*|[叔侄女儿子]+'
people = set()
qinshu_temp = pd.DataFrame(columns=['person1','person2','interval','detail','related_stk'],dtype=object)
qinshu_temp.to_csv('../data/股权及亲属关系/qinshu.csv',encoding='gb18030',index=None)

for i in qinshu.index:
    reltype = qinshu.loc[i,'S0601a']
    detail = qinshu.loc[i,'S0607a']
    if reltype==6 and detail is not np.nan:#reltype=6为亲属关系
        year = int(qinshu.loc[i,'Reptdt'].split('-')[0])
        person1 = qinshu.loc[i,'S0603a']
        person2 = qinshu.loc[i,'S0604a']
        keyword = re.findall(pattern,detail.replace('女士',''))
        if len(keyword)>0:
            people.add(person1)
            people.add(person2)
            qinshu_temp = pd.DataFrame(
                        {'person1':person1,
                        'person2':person2,
                        'interval':year,
                        'detail':detail,
                        'related_stk': qinshu.loc[i,'Stkcd']},index=[0])
                        # json.dumps(keyword,ensure_ascii=False)
            qinshu_temp.to_csv('../data/股权及亲属关系/qinshu.csv',encoding='gb18030',mode='a',header=None,index=None)
# re.findall(pattern,'潮州合众、潮州市炎城策划咨询有限公司的股东之一李湘娟为王建瑜的兄长的配偶'.replace('女士',''))

股权及亲属关系文件 里面的股东关系处理起来太麻烦了，不考虑了

In [4]:
'''
# 提取股权及亲属关系文件中的股东关系
gudong_temp = pd.DataFrame(columns=['holded','holder','interval','detail'],dtype=object)
gudong_temp.to_csv('../data/股权及亲属关系/gudong.csv',encoding='gb18030',index=None)

# 这里的关联只是关联方关系，并不形成关联交易

for i in qinshu.index:
    Stkcd = qinshu.loc[i,'Stkcd']
    year = int(qinshu.loc[i,'Reptdt'].split('-')[0])
    reltype = qinshu.loc[i,'S0601a']
    is_holded_stk_gudong = qinshu.loc[i,'S0602a']
    person1 = qinshu.loc[i,'S0603a']
    person2 = qinshu.loc[i,'S0604a']
    if is_holded_stk_gudong==1:
        gudong_temp = pd.DataFrame(
            {'holded':Stkcd,
            'holder':person1,
            'interval':year},index=[0])
        gudong_temp.to_csv('../data/股权及亲属关系/gudong.csv',encoding='gb18030',mode='a',header=None,index=None)

    is_holder_stk_gudong = qinshu.loc[i,'S0605a']
    if is_holder_stk_gudong==1:
        gudong_temp = pd.DataFrame(
            {'holded':Stkcd,
            'holder':person2,
            'interval':year},index=[0])
        gudong_temp.to_csv('../data/股权及亲属关系/gudong.csv',encoding='gb18030',mode='a',header=None,index=None)

    if reltype==1: # 持股关系
        detail = qinshu.loc[i,'S0607a']
        gudong_temp = pd.DataFrame(
            {'holded':person1,
            'holder':person2,
            'interval':year,
            'detail':detail},index=[0])
        gudong_temp.to_csv('../data/股权及亲属关系/gudong.csv',encoding='gb18030',mode='a',header=None,index=None)
    # elif reltype==2: # 关联关系
        
# 去重
gudong_temp = pd.read_csv('../data/股权及亲属关系/gudong.csv',encoding='gb18030')
gudong_temp.drop_duplicates(subset=['holded','holder','interval'],keep='first',inplace=True)
gudong_temp.to_csv('../data/股权及亲属关系/gudong.csv',encoding='gb18030',index=None)
'''

亲戚关系人名配对

In [10]:
# 亲戚关系
# '李中秋','吉祥','王萍','王玲玲','刘建民','陈小兵','刘畅','李巍','赵晓红','胡颖', 这些亲戚文件中的人在高管关系中是有重复的
print(duel_people_stk_id['李中秋'])
qinshu = pd.read_csv('../data/股权及亲属关系/qinshu.csv',encoding='gb18030')
qinshu.person1 = [name+'-'+str(duel_people_stk_id[name][stk]) if name in repeat_names and stk in duel_people_stk_id[name] else name for name, stk in zip(qinshu.person1, qinshu.related_stk)]
qinshu.person2 = [name+'-'+str(duel_people_stk_id[name][stk]) if name in repeat_names and stk in duel_people_stk_id[name] else name for name, stk in zip(qinshu.person2, qinshu.related_stk)]
qinshu.to_csv('../data/股权及亲属关系/qinshu_no_dup_person.csv',encoding='gb18030',index=None)

##### 子公司关系
对于公司名字的通用操作：将英文名的公司去重（删除标点、统一为大写、删除空格等），将中文名的公司名字中的半角标点转为全角，将公司全称和股票代码对上

In [74]:
subfirm = pd.read_excel('../data/上市公司子公司情况表/FN_Fn061.xlsx',encoding='gb18030')
subfirm.drop([0,1], axis=0, inplace=True )
subfirm.drop(['FN_Fn06109','FN_Fn06110'], axis=1, inplace=True)
subfirm.rename(columns={'EndDate':'interval','FN_Fn06101':'subsidiary'},inplace=True)
subfirm.drop_duplicates(subset=['Stkcd','subsidiary','interval'],keep='first',inplace=True)
subfirm = subfirm.reset_index(drop=True)

subfirm.interval = [int(t.split('-')[0]) for t in subfirm.interval]
# 把一些英文公司名字里面的,.和空格删掉
subfirm.subsidiary = [re.sub(r'）', ')', re.sub(r'（','(', re.sub(r'[,.，“”" \u3000]+','',i))).upper() for i in subfirm.subsidiary]
# 把全称和股票代码链接起来
subfirm.subsidiary = [name_id_dict[i] if i in name_id_dict else i for i in subfirm.subsidiary]
subfirm.to_csv('../data/上市公司子公司情况表/subsidiary_fullname_matched.csv',encoding='gb18030',index=None)

##### 关联交易

In [34]:
guanlian_1 = pd.read_excel('../data/关联方交易/RPT_Operation.xlsx',encoding='gb18030')
guanlian_2 = pd.read_excel('../data/关联方交易/RPT_Operation1.xlsx',encoding='gb18030')

guanlian_1.drop([0,1],inplace=True)
guanlian_2.drop([0,1],inplace=True)

guanlian = pd.concat([guanlian_1,guanlian_2])
guanlian = guanlian.reset_index(drop=True)


In [36]:
guanlian_temp = pd.DataFrame(columns=['seller','buyer','interval','trans_type'],dtype=object)
guanlian_temp.to_csv('../data/关联方交易/guanlian.csv',encoding='gb18030',index=None)

buy_pattern = r'[支代]付|购|信托产品|接受担保|接受[^A-Za-z0-9]*[劳服业务管理工程]+|承租|拆入|^[^发提]*[接受委托]*[贷借]款'
sell_pattern = r'[销出]售|转让|[提供]*担保|[代]*收[取]*|提供[^A-Za-z0-9]*[劳服业务管理工程]+|出租|拆[出借]+|承包|^[^委]*借款|[发放提供][贷借]款'
bi_pattern = r'互[相销]|共同'
useless_pattern = r'增资|技术服务|租赁|工程收入|资金往来|委托[经营利息]*|资产托管|还款|承销|资产置换|[劳务股权][交易托管]|装修|签署|抵款|抵押'

break_point = 0

for i in guanlian.index: 
    if i < break_point:
        continue
    # 交易日期，不是公告日期
    year = int(guanlian.loc[i,'Trddt'].split('-')[0])
    
    # 交易性质：1-该交易为本公司与关联方之间的交易;2-关联公司之间的交易
    trans_nature = guanlian.loc[i,'Trasub']
    # 交易方向：1=上市公司处于卖方立场，2=上市公司处于买方立场，3=上市公司处于无法分辨。
    trans_direct = guanlian.loc[i,'Direction']
    content = guanlian.loc[i,'Content']
    trans_type = guanlian.loc[i,'Kind']
    Stkcd = int(guanlian.loc[i,'Stkcd'])
    guanlian_firm = guanlian.loc[i,'Repart'].split(',')

    if trans_nature==1:
        if trans_direct == 1:
            seller,buyer = Stkcd, guanlian_firm[0]
        elif trans_direct==2:
            seller,buyer = guanlian_firm[0], Stkcd
        else: # trans_direct=3 多为共同执行
            seller,buyer = guanlian_firm[0], Stkcd

    elif trans_nature==2:
        #print(trans_type)
        if len(re.findall(buy_pattern, trans_type))>0:
            seller, buyer = guanlian_firm[-1], guanlian_firm[0]
        
        elif len(re.findall(sell_pattern, trans_type)) > 0:
            seller, buyer = guanlian_firm[0],guanlian_firm[-1]
        
        elif len(re.findall(bi_pattern, trans_type)) > 0: # 双方共同执行
            seller, buyer = guanlian_firm[0],guanlian_firm[-1]

        elif '供' in trans_type:
            try:
                firm_position = [content.find(j) for j in guanlian_firm]
            except:
                continue
            if firm_position[0] <= firm_position[-1]:
                seller, buyer = guanlian_firm[0],guanlian_firm[-1]
            else:
                seller, buyer = guanlian_firm[-1],guanlian_firm[0]
        elif len(re.findall(useless_pattern, trans_type)) > 0:
            continue
        else: # 其他pattern太多了，涵盖不完了
            continue
    else:
        continue
    
    guanlian_temp = pd.DataFrame({'seller':seller,
                'buyer':buyer,
                'interval':year,
                'trans_type':trans_type},index=[0])
    guanlian_temp.to_csv('../data/关联方交易/guanlian.csv',encoding='gb18030',mode='a',header=None,index=None)

    del seller, buyer, year


In [40]:
guanlian = pd.read_csv('../data/关联方交易/guanlian.csv',encoding='gb18030')
print(guanlian.shape)
# 因为构造guanlian.csv时没有去重，所以要在这里去重
guanlian.drop_duplicates(subset=['seller','buyer','interval'],keep='last',inplace=True)
# guanlian.reset_index(drop=True, inplace=True)
guanlian.to_csv('../data/关联方交易/guanlian.csv', encoding='gb18030', index=None)

(1371739, 4)


第二轮筛选，排除买/卖方指代不明的关联交易，以及对公司的名字（全角半角、英文名大小写和空格、删除标点）进行处理

In [219]:
guanlian = pd.read_csv('../data/关联方交易/guanlian.csv',encoding='gb18030')
print(guanlian.shape)

firm_except = r'[高管理]+人员|公司管理层|骨干|成员|员工|关联方|[董|监]事|其他|关联公司|薪酬|副总|[合联营国有关联]+企业|集团内企业|总工程师|总经理|[0-9]*家*[分子项目]+公司|[名个位][自然一致行动发起]+人|[名个位]+股东'
drop_list = []
useless_name = 'ltd. .ltd inc. limitedsp.zo.o. s.l. llc. s.a. 子公司 limited股东'

for i in guanlian.index:
    seller = guanlian.loc[i,'seller']
    buyer = guanlian.loc[i,'buyer']
    if str(seller)=='nan' or str(buyer)=='nan':
        drop_list.append(i)
        continue

    if type(seller)==str:
        if '等' in seller and len(re.findall(r'[高中]等|等级',seller)) == 0:
            guanlian.loc[i,'seller'] = seller.split('等')[0]
        elif '及' in seller:
            guanlian.loc[i,'seller'] = seller.split('及')[0]

        if len(re.findall(firm_except,seller))>0 or seller.lower() in useless_name:
            drop_list.append(i)
            continue

    if type(buyer)==str:
        if '等' in buyer and len(re.findall(r'[高中]等|等级',buyer))== 0:
            guanlian.loc[i,'buyer'] = buyer.split('等')[0]
        elif '及' in buyer:
            guanlian.loc[i,'buyer'] = buyer.split('及')[0]

        if len(re.findall(firm_except,buyer))>0 or buyer.lower() in useless_name:
            drop_list.append(i)

guanlian.drop(drop_list,inplace=True)

# 把半角（）替换为()、统一成大写、删除空格和标点
guanlian.seller = [re.sub(r'）', ')', re.sub(r'（','(', re.sub(r'[,.，“”" \u3000]+','',i))).upper() for i in guanlian.seller]
guanlian.buyer = [re.sub(r'）', ')', re.sub(r'（','(', re.sub(r'[,.，“”" \u3000]+','',i))).upper() for i in guanlian.buyer]

# 再次去重
guanlian.drop_duplicates(subset=['seller','buyer','interval'],keep='last',inplace=True)

# 把其中的上市公司（全名）转化为股票代码
guanlian.seller = [name_id_dict[i] if i in name_id_dict else i for i in guanlian.seller]
guanlian.buyer = [name_id_dict[i] if i in name_id_dict else i for i in guanlian.buyer]

guanlian.to_csv('../data/关联方交易/guanlian_fullname_matched.csv',encoding='gb18030',index=None)

(732925, 4)


人名配对，把重名的人的人名后面加上‘-id’

In [89]:
# 关联交易
guanlian = pd.read_csv('../data/关联方交易/guanlian_fullname_matched.csv',encoding='gb18030')

guanlian_temp = pd.DataFrame(columns=['seller','buyer','interval','trans_type'],dtype=object)
guanlian_temp.to_csv('../data/关联方交易/guanlian_no_dup_person.csv',encoding='gb18030',index=None)
for i in guanlian.index:
    seller, buyer = guanlian.loc[i,'seller'], guanlian.loc[i,'buyer']
    if type(buyer)==str and len(buyer)<=4 and buyer in repeat_names:
        # 识别 buyer 里面有重名的人名
        if re.match(r'[0-9]+',seller) is not None:
            seller = int(seller)
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'seller']) is not None:
            seller = int(guanlian.loc[i+1,'seller'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'seller']) is not None:
            seller = int(guanlian.loc[i-1,'seller'])
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'buyer']) is not None:
            seller = int(guanlian.loc[i+1,'buyer'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'buyer']) is not None:
            seller = int(guanlian.loc[i-1,'buyer'])

        if seller in duel_people_stk_id[buyer]:
            buyer = buyer + '-' + str(duel_people_stk_id[buyer][seller])
        # 还原seller
        seller = guanlian.loc[i,'seller']
        
    elif type(seller)==str and len(seller)<=4 and seller in repeat_names:
        # 识别 seller 里面有重名的人名
        if re.match(r'[0-9]+',buyer) is not None:
            buyer = int(buyer)
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'buyer']) is not None:
            buyer = int(guanlian.loc[i+1,'buyer'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'buyer']) is not None:
            buyer = int(guanlian.loc[i-1,'buyer'])
        elif re.match(r'[0-9]+',guanlian.loc[i+1,'seller']) is not None:
            buyer = int(guanlian.loc[i+1,'seller'])
        elif re.match(r'[0-9]+',guanlian.loc[i-1,'seller']) is not None:
            buyer = int(guanlian.loc[i-1,'seller'])
        # print(seller, buyer, type(buyer), buyer in duel_people_stk_id[seller])
        if buyer in duel_people_stk_id[seller]:
            seller = seller + '-' + str(duel_people_stk_id[seller][buyer])
        buyer = guanlian.loc[i,'buyer']
    
    guanlian_temp = pd.DataFrame({
        'seller': seller,
        'buyer': buyer,
        'interval': guanlian.loc[i,'interval'],
        'trans_type': guanlian.loc[i,'trans_type']}, index=[0])
    guanlian_temp.to_csv('../data/关联方交易/guanlian_no_dup_person.csv',encoding='gb18030',index=None, header=None, mode='a')


In [90]:
# 再次去重
guanlian = pd.read_csv('../data/关联方交易/guanlian_no_dup_person.csv',encoding='gb18030')
print(guanlian.shape)
guanlian.drop_duplicates(subset=['seller','buyer','interval'],keep='last',inplace=True)
guanlian.to_csv('../data/关联方交易/guanlian_no_dup_person.csv',encoding='gb18030',index=None)
print(guanlian.shape)

(687121, 4)
(686116, 4)


查看关联交易中存在哪些指代性的公司名字

In [92]:
guanlian = pd.read_csv('../data/关联方交易/guanlian_fullname_matched.csv',encoding='gb18030')
buhege = set()
for i in guanlian.index:
    seller = guanlian.loc[i,'seller']
    buyer = guanlian.loc[i,'buyer']
    try:
        seller = int(seller)
    except:
        if len(seller) < 6:
            buhege.add(seller)
    try:
        buyer = int(buyer)
    except:
        if len(buyer) < 6:
            buhege.add(buyer)
with open('../data/关联方交易/useless_list.json','w') as f:
    json.dump(list(buhege),f,ensure_ascii=False)
buhege

{'郝彭',
 '聂建林',
 '轴承厂',
 '梁兴安',
 '谢乐敏',
 '简伟文',
 '诸暨日报社',
 '中天仕名',
 '胡颖',
 '吴越俊',
 '氯碱发展',
 '何佳',
 '杭州安泽',
 '水泊峡公司',
 '朱雪松',
 '罗伶芝',
 '王小萍',
 '薛俊东',
 '蔡穗声',
 '薛华芬',
 '兰州环保厂',
 '周新龙',
 '李丽萍',
 '周玉梅',
 '裘坚樑',
 '费雨健',
 '仲玉容',
 '李素华',
 '马刚',
 '黄奕彬',
 '郭琦',
 '白建军',
 '张延',
 '佛山保恒',
 '雷伊实业',
 '高鸣',
 '刘绍喜',
 '许俏丹',
 '韩广德',
 '包春霞',
 '李中军',
 '冯会荣',
 '来新胜',
 '青岛广昌',
 '何启强',
 '龙功运',
 '张牧野',
 '张培德',
 '曹文法',
 '张伟泉',
 '中牧总公司',
 '龚锦娣',
 '杨耀光',
 '盐城邦尼',
 '胡常建',
 '何剑锋',
 '杨琪',
 '文宇',
 '蔡再华',
 '张雁冰',
 '陈永倬',
 '陈建鹏',
 '罗勤',
 '浙江精工',
 '乔鲁予',
 '高志伟先生',
 '陶国平',
 '操隆兵',
 '幸建平',
 '刘友生',
 '嘉兴诚鼎宏',
 '王跃先生',
 '徐建新',
 '赵强',
 '薛亮',
 '张云升',
 '西南兵工局',
 '邱光和',
 '陈先勇',
 '孙灵芝',
 '郑毓庆',
 '科龙空调',
 '孙奎发',
 '李金钟',
 '郑才刚',
 '广远公司',
 '天堂硅谷',
 '马焰',
 '李祖芹',
 '吴毅樑',
 '中望投资',
 '韦强',
 '陆平机器',
 '长虹日电',
 '恒大动力',
 '王青运',
 '郑有水',
 '王汉金',
 '铜陵物流',
 '广东特玻',
 '邵夕吾',
 '姜洪波',
 '重钢总医院',
 '栗工',
 '金敖大',
 '陈东',
 '翁小珏',
 '李建保',
 '周晓东',
 '金誉集团',
 '阳江中心坑',
 '潘岳燕',
 '西工厂',
 '何虎',
 '潇湘晨报社',
 '法国公司',
 '汽销鸿通',
 '郭英杰',
 '华黎',
 '钱木水',
 '徐伟成',
 '保利中山',


##### 股东关系

In [58]:
shareholder1 = pd.read_excel('../data/十大股东文件/HLD_Shareholders.xlsx',encoding='gb18030')
shareholder2 = pd.read_excel('../data/十大股东文件/HLD_Shareholders1.xlsx',encoding='gb18030')
# 初步处理
shareholder1.drop([0,1],inplace=True)
shareholder2.drop([0,1],inplace=True)
shareholder = pd.concat([shareholder1,shareholder2])
shareholder = shareholder.reset_index(drop=True)
print(shareholder.shape)

shareholder.Reptdt = [int(i.split('-')[0]) for i in shareholder.Reptdt]
shareholder.rename(columns={'Reptdt':'interval','S0301a':'shareholder'},inplace=True)
shareholder.drop(['S0304a','S0305a','S0306a','ShareholderNature','UnCriculationShares','CategoryID','Category'], axis=1, inplace=True)
# 十大股东是季度的数据，要去重
shareholder.drop_duplicates(subset=['Stkcd','shareholder','interval'],keep='last',inplace=True)
shareholder.to_csv('../data/十大股东文件/shareholder.csv',encoding='gb18030',index=None)
print(shareholder.shape)

(1552557, 10)


处理公司名字

In [223]:
shareholder = pd.read_csv('../data/十大股东文件/shareholder.csv',encoding='gb18030')
print(shareholder.shape)
# 把 Ltd. inc. -> Ltd inc、半角替换为全角, \u3000 是空格
shareholder.shareholder = [re.sub(r'）', ')', re.sub(r'（','(', re.sub(r'[,.，“”" \u3000]+','',i))).upper() for i in shareholder.shareholder]
shareholder.shareholder = [re.sub(r'－', '-', i).split('-')[0] for i in shareholder.shareholder]

shareholder.shareholder = [i.split('等')[0] if '等' in i[1:] and len(re.findall(r'[高中]等|等[级权]',i)) == 0 else i for i in shareholder.shareholder]
shareholder.shareholder = [i.split('及')[0] if '及' in i[1:] else i for i in shareholder.shareholder] # 及不是第一个字

firm_except = r'[高级管理]+人员|骨干|成员|其他|[名个位][自然一致行动发起]+人|[名个位]+股东'
shareholder.drop(set([i if len(re.findall(firm_except,shareholder.shareholder[i])) > 0 else -1 for i in shareholder.index]) - set([-1]), axis = 0, inplace = True)

shareholder.shareholder = [name_id_dict[i] if i in name_id_dict else i for i in shareholder.shareholder]

print(shareholder.shape)
shareholder.drop_duplicates(subset=['Stkcd', 'interval', 'shareholder'],keep='first',inplace=True)
print(shareholder.shape)

shareholder.to_csv('../data/十大股东文件/shareholder_fullname_matched.csv',encoding='gb18030',index=None)

(740650, 3)
(740645, 3)
(699797, 3)


处理股东里重名的人名

In [224]:
shareholder = pd.read_csv('../data/十大股东文件/shareholder_fullname_matched.csv',encoding='gb18030')
shareholder.shareholder = [name+'-'+str(duel_people_stk_id[name][stk]) if name in repeat_names and stk in duel_people_stk_id[name] else name for name, stk in zip(shareholder.shareholder, shareholder.Stkcd)]
shareholder.to_csv('../data/十大股东文件/gudong_no_dup_person.csv',encoding='gb18030',index=None)

去除金融公司类别的大股东

In [226]:
shareholder = pd.read_csv('../data/十大股东文件/gudong_no_dup_person.csv',encoding='gb18030')
fin_pattern = r'基金|证券|银行|保险|[中农工建]行|财富|国泰君安|申银万国|国际金融|资管|易方达|资产管理|信托|理财'
print(shareholder.shape)
shareholder.drop(set([i if len(re.findall(fin_pattern, str(shareholder.shareholder[i]))) > 0 else -1 for i in shareholder.index]) - set([-1]), axis = 0, inplace = True)
print(shareholder.shape)
shareholder.to_csv('../data/十大股东文件/gudong_not_fin.csv',encoding='gb18030',index=None)

(699797, 3)
(540138, 3)


##### 供应商关系, 暂时不需要

In [ ]:

'''gongying = pd.read_excel('../data/前五大供应商关系/SC_TopFivePurchaseInfo_raw.xlsx',encoding='gb18030')
gongying.drop([0,1],axis=0,inplace=True)
gongying = gongying.reset_index(drop=True)
# gongying.set_index(['Symbol'],inplace=True)
gongying.set_index(['InstitutionName'],inplace=True)
print(gongying.shape)

# 先去掉一些看不出来公司名字的
import encodings
rule = r'第*[一二三四五A-Z]*供[应货方商]-*[一二三四五六七八九十a-eA-Z0-9]*|[A-E甲乙丙丁戊]+[有限]*公司|公司[一二三四五0-9]+|合计|第[一二三四五0-9]+[名位]|单位-*[0-9A-Z一二三四五]+|法人[一二三四五]*|客户[一二三四五0-9]*|排名[1-5]*|品牌[0-9A-Z]*|医药商[一二三四五]*|企业[A-Z]+'
drop_index = set()
for index in set(gongying.index):
    if len(re.findall(rule,index))>0:
        drop_index.add(index)
    elif len(index)==1 and len(re.findall(r'[0-9A-Z甲乙丙丁戊]',index))>0: # 名字就叫 A,B,C...1,2,3,4...
        drop_index.add(index)
    elif (len(index)>=1 or len(index)<=2) and len(re.findall(r'[A-Z][0-9]',index))>0: #A1,B2
        drop_index.add(index)
    elif '*' in index or '某公司' in index or '同一控制下' in index: # 曹**, ****技术有限公司
        drop_index.add(index)

gongying2 = gongying.drop(list(drop_index),axis=0,inplace=False)
gongying2.to_csv('前五大供应商关系/SC_TopFivePurchaseInfo.csv',encoding='gb18030')

# 生成用于组成KG的txt文件
gongying_temp = pd.DataFrame(columns=['Stkcd','gongying_firm','interval'])

gongying_temp.to_csv('../data/前五大供应商关系/gongying.csv',encoding='gb18030')

gongying2 = pd.read_csv('../data/前五大供应商关系/SC_TopFivePurchaseInfo.csv',encoding='gb18030')
for i in gongying2.index:
    symbol = gongying2.loc[i,'Symbol']
    year = int(gongying2.loc[i,'EndDate'].split('/')[0])
    if not gongying2.loc[i,'BusinessSymbol'] is np.nan:
        gongying_firm = gongying2.loc[i,'BusinessSymbol'].split(';')[0]
    else:
        gongying_firm = gongying2.loc[i,'InstitutionName']
    gongying_temp = pd.DataFrame(
        {'Stkcd':symbol,
        'gongying_firm':gongying_firm,
        'interval':year},index=[0])
    gongying_temp.to_csv('../data/前五大供应商关系/gongying.csv',encoding='gb18030',mode='a',header=None,index=None)'''